In [1]:
import copy
import multiprocessing as mp
import multiprocessing.dummy as dummy_mp
import os
import re
from datetime import datetime
from string import Template

import pymorphy2
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

FB_LOGIN = '89026051080'
FB_PASSWORD = 'z6A-8Wd-XiV-VNY'
login, password = FB_LOGIN, FB_PASSWORD

__base_url = 'https://www.facebook.com'
__base_group_url = 'https://www.facebook.com/groups'
__login_url = 'https://www.facebook.com/login'

options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-gpu")

options.add_argument("--disable-software-rasterizer")  # maybe not needed
options.add_argument("--disable-dev-shm-usage")  # maybe not needed

browser = webdriver.Chrome(options = options)

In [2]:
def enter_facebook(browser, login, password):
    """Login to fb

    :param login: (str) fb login
    :param password: (str) fb password

    :return: -1 if error
    """
    browser.get(__login_url)

    try:
        login_form = browser.find_element_by_xpath("//input[@id='email' or @name='email']")
        login_form.clear()
        login_form.send_keys(login)
    except Exception as e:
        print(e)
        print('error: email')
        return -1

    try:
        pass_form = browser.find_element_by_xpath("//input[@id='pass' or @name='pass']")
        pass_form.clear()
        pass_form.send_keys(password)
    except Exception as e:
        print(e)
        print('error: password')
        return -1

    try:
        browser.find_element_by_xpath("//button[@id='loginbutton']").click()
    except Exception as e:
        print(e)
        print('error: login button')
        return -1

    if 'login_attempt' in browser.current_url:
        return -1

In [3]:
enter_facebook(browser, FB_LOGIN, FB_PASSWORD)

In [4]:
# posts = fb.get_posts('100032357613324')
page_id = 'groups/vesti24russia'
browser.get(__base_url + '/' + page_id)

page_soup = BeautifulSoup(browser.page_source, 'lxml')

In [15]:
nonavbl = page_soup.find('title', text=re.compile('.*Страница не найдена.*'))
if nonavbl:
    print('Страница не найдена')
nonavbl = page_soup.find('h2', text=re.compile('.*Эта страница недоступна.*'))
if nonavbl:
    print('Эта страница недоступна')
nonavbl = page_soup.find('h2', text=re.compile('.*К сожалению, этот контент сейчас недоступен.*'))
if nonavbl:
    print('К сожалению, этот контент сейчас недоступен')

In [16]:
page_soup.find('title', text=re.compile('.*Страница не найдена.*'))

### Блок для замены функции поиска постов

In [5]:
from time import sleep 
from datetime import datetime, timedelta
from selenium.webdriver.common.action_chains import ActionChains
actions = ActionChains(browser)

In [6]:
date1 = '24 марта в 10:20'
date2 = '1 ч.'
date3 = '11 февраля'
date4 = '2 д.'
date5 = '30 мин.'

textToDate_months = {
    'января': 1,
    'февраля': 2,
    'марта': 3,
    'фпреля': 4,
    'мая': 5,
    'июня': 6,
    'июля': 7,
    'августа': 8,
    'сентября': 9,
    'октября': 10,
    'ноября': 11,
    'декабря': 12
}

def textToDate(text):        
    date = datetime.today()
    if text:  
        s = text.split()
        t = int(s[0])
        if 'ч.' in s[1]:
            date -= timedelta(hours=t)
        elif 'д.' in s[1]:
            date -= timedelta(days=t)
        elif 'мин.' in s[1]:
            date -= timedelta(minutes=t)
        else:
            date = datetime(date.year, int(textToDate_months.get(s[1], date.month)), t)                
    return date
    

[textToDate(d) for d in [date1, date2, date3, date4, date5]]

[datetime.datetime(2021, 3, 24, 0, 0),
 datetime.datetime(2021, 4, 11, 19, 33, 54, 356899),
 datetime.datetime(2021, 2, 11, 0, 0),
 datetime.datetime(2021, 4, 9, 20, 33, 54, 356899),
 datetime.datetime(2021, 4, 11, 20, 3, 54, 356899)]

In [7]:
def getPostDate(browser, post):
    browser.execute_script("arguments[0].scrollIntoView();", post)
    abilities = post.find_elements_by_xpath('.//span/a/span')
    if len(abilities)> 0:
        return textToDate(abilities[0].text)
    return datetime.today()            

In [197]:
# try:
#     WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.TAG_NAME, "div[role='article']")))
# except TimeoutException:
#     print(posts_list) 

In [9]:
from_date = datetime.today() - timedelta(days=2)
from_date = None

In [10]:
limit = 20
postcount = -1
posts_list = []
error_count = 0
mindate = datetime.today()

while ((not from_date and len(posts_list) < limit) or (from_date and mindate >= from_date)) and len(posts_list) != postcount:
    postcount = len(posts_list)
    for i in range(postcount // 3 + 1):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(0.5)    
    posts_list = browser.find_elements_by_tag_name("div[role='article'][aria-posinset]")
    if len(posts_list)>0:
        mindate = getPostDate(browser, posts_list[-1])
    print(len(posts_list), postcount)

if from_date:
    posts_list = [post for post in posts_list if getPostDate(browser, post) >= from_date]
    
print(len(posts_list), postcount)

4 0
7 4
16 7
31 16
31 16


In [11]:
posts_xpath = "div[role='article'][aria-posinset]"
post_more_button = './/div[contains(text(), "Ещё")]'
post_comment_more_button = './/span[starts-with(text(), "Посмотреть ещё")]'
post_comment_more_button2 = './/span[starts-with(text(), "Показать ещё")]'
need_comments = True

result = []
for ind, post in enumerate(posts_list):
    try:
        stage = 'Scroll to post and date'
        postDate = getPostDate(browser, post)
        # раскрыть ещё, если есть
        if not from_date or postDate >= from_date:

            stage = 'Open comments'
            # раскрыть все комменты, если надо
            if need_comments:
                more_comments_finder = lambda: post.find_elements_by_xpath(
                    post_comment_more_button) + post.find_elements_by_xpath(
                    post_comment_more_button2)
                more_comments = more_comments_finder()                
                while more_comments:    
                    stage += "|"                
                    more_comments[0].click()
                    sleep(1)
                    more_comments = more_comments_finder()                    
            
            stage = 'More buttons'    
            more = post.find_elements_by_xpath(post_more_button)
            while more:
                stage += "|"
                more[0].click()
                more = post.find_elements_by_xpath(post_more_button)
                
            result.append(BeautifulSoup(post.get_attribute('innerHTML'), 'lxml'))
    except Exception as e:
        print(f'Пост {ind + 1} пропущен. Этап: {stage}\n', e)

Пост 8 пропущен. Этап: More buttons|
 Message: element click intercepted: Element <div class="oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl oo9gr5id gpro0wi8 lrazzd5p" role="button" tabindex="0">...</div> is not clickable at point (713, 10). Other element would receive the click: <ul class="thodolrn ojvp67qx taijpn5t buofh1pr j83agx80 aovydwv3 bqdfd6uv">...</ul>
  (Session info: chrome=89.0.4389.114)

Пост 19 пропущен. Этап: More buttons|
 Message: element click intercepted: Element <div class="oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl oo9gr5id gpro0wi8 lrazzd5p" role="button" tabindex="0">...</div> is not clickable at point (688, 10). Other element would receive the click: <ul class

In [38]:
post = posts_list[16]

In [14]:
browser.execute_script("arguments[0].scrollIntoView();", post)
more = post.find_elements_by_xpath('.//div[contains(text(), "Ещё")]')
if more:
    more[0].click()

In [15]:
   
more_comments_finder = lambda: post.find_elements_by_xpath('.//span[starts-with(text(), "Посмотреть ещё")]') + post.find_elements_by_xpath('.//span[starts-with(text(), "Показать ещё")]')
more_comments = more_comments_finder()
while more_comments:
    more_comments[0].click()
    sleep(1)
    more_comments = more_comments_finder()    
    
comments = post.find_elements_by_tag_name("ul")[1]

In [35]:
len(post.find_elements_by_tag_name("ul"))

28

In [ ]:
print(len(posts_list))
dates = [getPostDate(browser, post) for post in posts_list]
print(len(dates))
dates
# post.get_attribute('innerHTML')

## Конструктор поста

In [12]:
def get_message(post_soup):
	try:
		content = post_soup.find('div', attrs={'dir': 'auto'}).find('span',
																	attrs={'dir': 'auto'}).findAll('div',
																								   recursive=False)
		return '\n'.join(list(map(lambda x: x.text, content)))
	except Exception as e:
		print(e)
		print('crashed while reading message')
		return ''

def get_id(post_soup):
	try:
        try:
            post_url = post_soup.find('a', attrs={'href': re.compile(r'.*/posts/.*')})['href']
            return re.search(r'/posts/([0-9]+)', post_url).group(1)
        except:
            post_url = post_soup.find('a', attrs={'href': re.compile(r'.*/permalink/.*')})['href']    
            return re.search(r'/permalink/([0-9]+)', post_url).group(1) 
	except Exception as e:
		print(e)
		print('crashed while reading id')
		# return ''
		pass

	try:
		img_url = post_soup.find('a', attrs={'href': re.compile(r'.*/photos/[a-z]*\.[0-9]+/[0-9]+/')})['href']
		return re.search(r'/photos/[a-z]+\.[0-9]+/([0-9]+)/', img_url).group(1)
	except Exception as e:
		print(e)
		pass

	try:
		permalink = post_soup.find('a', attrs={'href': re.compile(r'.*/permalink/[0-9]+/.*')})['href']
		return re.search(r'.*/permalink/([0-9]+)/.*', permalink).group(1)
	except Exception as e:
		print(e)
		return ''

def get_icon_url(post_soup):
	try:
		return post_soup.find('image')['xlink:href']
	except Exception as e:
		print(e)
		print('crashed while reading comment icon')
		return None

def get_date(post_soup):
	try:
		date = post_soup.findAll(
			'a',
			attrs={
				'role': 'link',
				'tabindex': '0',
				'aria-label': re.compile(r'.*')
			})[1].text
		return textToDate(date)
	except Exception as e:
		print(e)
		print('crashed while searching date')
		return None

def get_comments_count(post_soup):
	try:
		coments_count = post_soup.find('span', text=re.compile(r'^\s*Комментарии:\s*\d+\s*$'))
		if coments_count:
			return int(re.search(r'\d+', coments_count.text).group(0))
	except Exception as e:
		print(e)
		print('crashed while searching number of comments')
	return 0

def get_likes(post_soup):
	try:
		likes = post_soup.find(
			'span',
			attrs={'role': 'toolbar'}
		).parent.find(
			'div',
			recursive=False
		).span.div.span.text
		return int(likes)
	except Exception as e:
		print(e)
		print('crashed while counting likes')
		return 0

def get_reposts_count(post_soup):
	try:
		coments_count = post_soup.find('span', text=re.compile(r'^\s*Поделились:\s*\d+\s*$'))
		if coments_count:
			return int(re.search(r'\d+', coments_count.text).group(0))
	except Exception as e:
		print(e)
		print('crashed while searching number of reposts')
	return 0

def get_comments(post_soup):
	try:
		comments_soup = post_soup.findAll('div', attrs={'role': 'article'})
		return [comm for comm in [Comment(com_el) for com_el in comments_soup] if comm is not None]
	except Exception as e:
		print(e)
		print('crashed while searching comments')
		return []

In [13]:
post_soup = result[2]

In [21]:
try:
    post_url = post_soup.find('a', attrs={'href': re.compile(r'.*/posts/.*')})['href']
    re.search(r'/posts/([0-9]+)', post_url).group(1)
except:
    post_url = post_soup.find('a', attrs={'href': re.compile(r'.*/permalink/.*')})['href']    
    re.search(r'/permalink/([0-9]+)', post_url).group(1)

In [51]:

class Comment:

    def get_id(self, comment_soup):
        """Get comment's ID

        :param comment_soup: (object) comment soup

        :return: (str) comment's ID (None if error)
        """
        try:
            comment_url = comment_soup.find('a', attrs={'href': re.compile('.*\?comment_id=[0-9]+&.*')})['href']
            return re.search('[0-9]+', re.search('comment_id=[0-9]+', comment_url).group(0)).group(0)
        except Exception as e:
            print(e)
            print('crashed while reading comment id')
            return None

    def get_icon_url(self, comment_soup):
        """Get comment author's avatar

        :param comment_soup: (object) comment soup

        :return: (str) author's avatar url (None if error)
        """
        try:
            return comment_soup.find('image')['xlink:href']
        except Exception as e:
            print(e)
            print('crashed while reading comment icon')
            return None

    def get_date(self, comment_soup):
        """Get comment's date

        :param comment_soup: (object) comment soup

        :return: (int) Unix Timestamp (None if error)
        """
        try:
            date = comment_soup.findAll('a', attrs={'role': 'link', 'tabindex': '0'})[1].text
            return textToDate(date)
        except Exception as e:
            print(e)
            print('crashed while searching date im comment')
            return None

    def get_message(self, comment_soup):
        """Get comment's text

        :param comment_soup: (object) comment soup

        :return: (str) comment's text (None if error)
        """
        try:
            content = comment_soup.findAll('div', attrs={'dir': 'auto'})
            return '\n'.join([x.text.strip() for x in content])
        except Exception as e:
            print(e)
            print('crashed while reading comment')
            return None

    def get_owner_id(self, comment_soup):
        """Get comment author's ID

        :param comment_soup: (object) comment soup

        :return: (str) author's ID (None if error)
        """
        try:
            link = comment_soup.a['href']
            id_v1 = re.search('profile.php\?id=[0-9]+&', link)
            if id_v1:
                return re.search('[0-9]+', id_v1.group(0)).group(0)
            else:
                return re.search('facebook.com/.*\?comment_id', link).group(0).replace('facebook.com/', "").replace(
                    '?comment_id', "")
        except Exception as e:
            print(e)
            print('crashed while searching comment owner id')
            return None

    def __init__(self, comment_soup):
        """Initialize

        :attr comment_soup: (object) comment soup
        """
        self.soup = comment_soup
        self.id = self.get_id(comment_soup)
        self.icon = self.get_icon_url(comment_soup)
        self.date = self.get_date(self.soup)
        self.owner = self.get_owner_id(comment_soup)
        self.message = self.get_message(comment_soup)

    def to_dict(self):
        """Format to dict

        :return: (dict) comment
        """
        return {'id': self.id,
                'avatar': self.icon,
                'owner': self.owner,
                'date': (None if self.date is None else self.date.timestamp()),
                'text': self.message}

In [22]:
post_soup = result[2]
print(get_comments_count(post_soup))
coms = post_soup.findAll('div', attrs={'role': 'article'})
comment_soup = coms[0]

3


In [30]:
for post_soup in result:
    comments_soup = post_soup.findAll('div', attrs={'role': 'article'})
    for comment_soup in comments_soup:
        print(comment_soup.text)
        link = comment_soup.a['href']
        id_v1 = re.search('profile.php\?id=[0-9]+&', link)
        if id_v1:
            ret = re.search('[0-9]+',id_v1.group(0)).group(0)
        else:
            ret = re.search('facebook.com/.*\?', link).group(0).replace('facebook.com/',"").replace('?comment_id',"")
        ret

Иван Сцарьhttps://www.facebook.com/hesenlibehruz81/videos/1218981971833217/?sfnsn=mo · Нравится · 2 ч.
Капітан Мишлаєвськийhttps://youtu.be/98RpQ2hKKFIYOUTUBE.COMТы сам понял что сказалТы сам понял что сказал · Нравится · 1 д.
Андрей Устиновhttps://i.mycdn.me/i...I.MYCDN.MEi.mycdn.mei.mycdn.me · Нравится · 1 д.
Иван Сцарьhttps://www.facebook.com/hesenlibehruz81/videos/1218981971833217/?sfnsn=mo · Нравится · 3 ч.
Rifat NizamowМальчик хочет Тамбов....Приостановить GIF-анимациюGIPHY · Нравится · 1 д.
Расим МавридиМало , а на зарплаты сотрудникам сколько выделили ? По 15000 на на 20 человек ? · Нравится · 3 ч.
Федя МанинЗаявлятель. Иди сельским хозяйством занимайся, потомственный аграрий2 · Нравится · 3 д.
Poli DjavelБиологическое оружие было использовано в Китае как короновирус. А затем во всем мире. Было использовано глобалистами-демократами США. Об этом надо говорить4 · Нравится · 3 д.
Edouard ColaniPoli Djavel Здравствуйте, извините меня за это внезапное вторжение в вашу частную жизнь.

In [29]:
comment_soup.a['href']

'https://www.facebook.com/andrey.ustinov.12?__cft__[0]=AZWrdwqvVkCDtmzDUL_DWhkx-1-iuakIsIzM5uUTL5Vgp72sV7FasomZ4GIvsb-SD3heABzvzcimR-rWLLwbruS9XNEuyBOF_ZFusidZvj8jqTD1hrESRxYslnej3c1zo9kDUyU_Ln_BVlA141odonirFWwpu7pgMD5xizlv3TIwowTkX5086TzTw_svy290mqQ&__tn__=R]-R'

In [23]:
def get_id(comment_soup):
    try:
        comment_url = comment_soup.find('a', attrs={'href': re.compile('.*\?comment_id=[0-9]+&.*')})['href']
        return re.search('[0-9]+', re.search('comment_id=[0-9]+', comment_url).group(0)).group(0)
    except Exception as e:
        print(e)
        print('crashed while reading comment id')
        return None

def get_icon_url(comment_soup):
    try:
        return comment_soup.find('image')['xlink:href']
    except Exception as e:
        print(e)
        print('crashed while reading comment icon')
        return None

def get_date(comment_soup):
    try:
        date = comment_soup.findAll('a', attrs={'role': 'link','tabindex': '0'})[-1].text 
        return textToDate(date)
    except Exception as e:
        print(e)
        print('crashed while searching date')
        return None
    print('fe')

def get_message(comment_soup):
    try:
        content = comment_soup.findAll('div', attrs={'dir': 'auto'}) 
        return '\n'.join([x.text.strip() for x in content])
    except Exception as e:
        print(e)
        print('crashed while reading comment')
        return None

def get_owner_id(comment_soup):
    try:    
        link = comment_soup.a['href']
        id_v1 = re.search('profile.php\?id=[0-9]+&', link)
        if id_v1:
            return re.search('[0-9]+',id_v1.group(0)).group(0)
        else:
            return re.search('facebook.com/.*\?comment_id', link).group(0).replace('facebook.com/',"").replace('?comment_id',"")
    except Exception as e:
        print(e)
        print('crashed while searching comment owner id')
        return None

In [575]:
link ="https://www.facebook.com/riri.nekke?comment_id=Y29tbWVudDoxOTY0NTQ3ODMzNjk5OTYyXzE5NjQ1NTU2MDcwMzI1MTg%3D&amp;__cft__[0]=AZVFE8FNFBElGB8j3nvVBlou4kh2wpsH2v1ABI11uhvTNe2gOciMzWdRdxEYj0YnoAxUFKNxXl8gt7jOuS3r8Kom4TlzKvgaP5TF8xnCihdM1RIWSSeAZszPCusX-qfKZ6DvuG2zyzU0IkN5Az59w6Om&amp;__tn__=R]-R"

id_v2 = re.search('facebook.com/.*\?comment_id', link).group(0).replace('facebook.com/',"").replace('?comment_id',"")
id_v2
# if id_v1:
#     print(re.search('[0-9]+',id_v1.group(0)))

'riri.nekke'

In [579]:
for comment_soup in coms:
    link = comment_soup.a['href']
    id_v1 = re.search('profile.php\?id=[0-9]+&', link)
    if id_v1:
        print(re.search('[0-9]+',id_v1.group(0)).group(0))
    else:
        print(re.search('facebook.com/.*\?comment_id', link).group(0).replace('facebook.com/',"").replace('?comment_id',""))
            
    

riri.nekke
roma.roman.7
oligarhist
serdjio.vladimirovich


In [581]:
for comment_soup in coms:
    print(get_id(comment_soup))
    print( get_icon_url(comment_soup))
    print( get_date(comment_soup))
    print( get_owner_id(comment_soup))
    print( get_message(comment_soup))
    print('-' * 20)

1964555607032518
https://scontent.fhel6-1.fna.fbcdn.net/v/t1.6435-1/cp0/p32x32/168495768_115357750649642_8084621296399784489_n.jpg?_nc_cat=103&ccb=1-3&_nc_sid=7206a8&_nc_ohc=NtBzOIYF2m0AX-v_IfL&_nc_ht=scontent.fhel6-1.fna&tp=27&oh=1823961927c57bfb70ff2bc6193b00db&oe=609920E2
2021-04-11 01:20:08.178626
riri.nekke
зато вино урожая 2021 будет дорогим. Не переживайте за французских виноделов, Медуза, с ними все будет ок..
--------------------
1964558527032226
https://scontent-hel3-1.xx.fbcdn.net/v/t1.6435-1/cp0/c5.0.32.32a/p32x32/78980262_2994341460585815_5069468285947346944_n.jpg?_nc_cat=106&ccb=1-3&_nc_sid=7206a8&_nc_ohc=Ex5CXBeXNqcAX_EFUjc&_nc_ht=scontent-hel3-1.xx&tp=27&oh=231d2547ce69afa68f10e094e4f44214&oe=60967F13
2021-04-11 01:26:08.179634
roma.roman.7
Сколько мазута сожгли и жгут от этого ещё хуже атмосферу
--------------------
1964559973698748
https://scontent-hel3-1.xx.fbcdn.net/v/t1.18169-1/cp0/p32x32/14691143_1192772337432366_7966973878949592162_n.jpg?_nc_cat=106&ccb=1-3&_nc_s

In [70]:
for post_soup in result:
    comments_soup = post_soup.findAll('div', attrs={'role': 'article'})
    for comment_soup in comments_soup:
        date = comment_soup.findAll('a', attrs={'role': 'link','tabindex': '0'})[-1].text 
        print(date)
        print(textToDate(date))

51 мин.
2021-04-11 17:50:01.460948
3 ч.
2021-04-11 15:41:01.487933
2 ч.
2021-04-11 16:41:01.487933
2 ч.
2021-04-11 16:41:01.488932
2 ч.
2021-04-11 16:41:01.490932
3 ч.
2021-04-11 15:41:01.492931
3 ч.
2021-04-11 15:41:01.493930
1 ч.
2021-04-11 17:41:01.493930
1 ч.
2021-04-11 17:41:01.494929
1 ч.
2021-04-11 17:41:01.495929
2 ч.
2021-04-11 16:41:01.497927
3 ч.
2021-04-11 15:41:01.498927
3 ч.
2021-04-11 15:41:01.499927
3 ч.
2021-04-11 15:41:01.500926
19 мин.
2021-04-11 18:22:01.501925
2 ч.
2021-04-11 16:41:01.502924
2 ч.
2021-04-11 16:41:01.502924
2 ч.
2021-04-11 16:41:01.508921
3 ч.
2021-04-11 15:41:01.509922
3 ч.
2021-04-11 15:41:01.510922
1 ч.
2021-04-11 17:41:01.511919
1 ч.
2021-04-11 17:41:01.513920
3 ч.
2021-04-11 15:41:01.514920
3 ч.
2021-04-11 15:41:01.515917
2 ч.
2021-04-11 16:41:01.516916
2 ч.
2021-04-11 16:41:01.517915
2 ч.
2021-04-11 16:41:01.517915
2 ч.
2021-04-11 16:41:01.518914
4 ч.
2021-04-11 14:41:01.525911
5 ч.
2021-04-11 13:41:01.549901
5 ч.
2021-04-11 13:41:01.551896
2 

In [65]:
for i in comment_soup.findAll('a', attrs={'role': 'link','tabindex': '0'}):
    print(i.text)
    


Андрей Остроушко
https://novosti-n.org/.../-Nakurennij-voennosluzhashhyj...
«Накуренный» военнослужащий выстрелил себе в голову из автомата. Видео 18+
1 ч.
